In [1]:
import configparser
from datetime import datetime, timedelta
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, dayofweek
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import DateType,StringType
from pyspark.sql import functions as F
from pyspark.sql.functions import avg
import pandas as pd

In [2]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

Exception: Java gateway process exited before sending its port number

In [3]:
print(spark)

NameError: name 'spark' is not defined

In [4]:
my_spark = SparkSession.builder.getOrCreate()

Exception: Java gateway process exited before sending its port number

In [1]:
from pyspark.sql import SparkSession

In [2]:
my_spark = SparkSession.builder.getOrCreate()

In [5]:
demographics_df = my_spark.read.option("multiline","true").json('us-cities-demographics.json')

In [6]:
demographics_df.show()

+----------------------+----------------+------+-----------------+------------+---------------+----------+------------------+--------------------+--------------+----------+----------------+
|Average Household Size|            City| Count|Female Population|Foreign-born|Male Population|Median Age|Number of Veterans|                Race|         State|State Code|Total Population|
+----------------------+----------------+------+-----------------+------------+---------------+----------+------------------+--------------------+--------------+----------+----------------+
|                   2.6|   Silver Spring| 25924|            41862|       30908|          40601|      33.8|              1562|  Hispanic or Latino|      Maryland|        MD|           82463|
|                  2.39|          Quincy| 58723|            49500|       32935|          44129|      41.0|              4147|               White| Massachusetts|        MA|           93629|
|                  2.58|          Hoover|  4759|  

In [7]:
demographics_df = demographics_df.dropna(subset=['Total Population'])

In [8]:
    demographics_df = demographics_df.drop_duplicates(subset=['City', 'State', 'Race'])


In [23]:
temperatures_df = my_spark.read.csv('temtest.csv',header=True)

In [24]:
temperatures_df.show()

+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|   City|      Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|1820-01-01|2.1010000000000004|                        3.217|Abilene|United States|  32.95N|  100.53W|
|1820-02-01|             6.926|                        2.853|Abilene|United States|  32.95N|  100.53W|
|1820-03-01|            10.767|                        2.395|Abilene|United States|  32.95N|  100.53W|
|1820-04-01|17.988999999999994|                        2.202|Abilene|United States|  32.95N|  100.53W|
|1820-05-01|            21.809|                        2.036|Abilene|United States|  32.95N|  100.53W|
|1820-06-01|            25.682|                        2.008|Abilene|United States|  32.95N|  100.53W|
|1820-07-01|            26.268|           1.8019999999999998|Abilene|Unit

In [25]:
    temperatures_df = temperatures_df.dropna(subset=['AverageTemperature'])


In [29]:
temperatures_df.show()

+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|   City|      Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-------+-------------+--------+---------+
|1820-05-01|            21.809|                        2.036|Abilene|United States|  32.95N|  100.53W|
|1820-10-01|             15.83|                        2.169|Abilene|United States|  32.95N|  100.53W|
|1820-03-01|            10.767|                        2.395|Abilene|United States|  32.95N|  100.53W|
|1820-01-01|2.1010000000000004|                        3.217|Abilene|United States|  32.95N|  100.53W|
|1821-03-01|            11.728|           2.6910000000000003|Abilene|United States|  32.95N|  100.53W|
|1821-04-01|            15.461|                        2.178|Abilene|United States|  32.95N|  100.53W|
|1820-08-01|            25.048|                        1.895|Abilene|Unit

In [27]:
temperatures_df = temperatures_df.drop_duplicates(subset=['dt', 'City', 'Country'])
temperatures_df = temperatures_df.filter(temperatures_df.Country == 'United States')


In [28]:
temperatures_df = temperatures_df.filter(temperatures_df.Country == 'United States')


In [32]:
time_table = temperatures_df.select(
        col('dt').alias('arrival_date'),
        dayofmonth('dt').alias('day'),
        weekofyear('dt').alias('week'),
        month('dt').alias('month'),
        year('dt').alias('year'),
        dayofweek('dt').alias('weekday')
    ) 

In [31]:
import configparser
from datetime import datetime, timedelta
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, dayofweek
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.types import DateType,StringType
from pyspark.sql import functions as F
from pyspark.sql.functions import avg

In [33]:
time_table.show()

+------------+---+----+-----+----+-------+
|arrival_date|day|week|month|year|weekday|
+------------+---+----+-----+----+-------+
|  1820-05-01|  1|  18|    5|1820|      2|
|  1820-10-01|  1|  39|   10|1820|      1|
|  1820-03-01|  1|   9|    3|1820|      4|
|  1820-01-01|  1|  52|    1|1820|      7|
|  1821-03-01|  1|   9|    3|1821|      5|
|  1821-04-01|  1|  13|    4|1821|      1|
|  1820-08-01|  1|  31|    8|1820|      3|
|  1820-09-01|  1|  35|    9|1820|      6|
|  1820-07-01|  1|  26|    7|1820|      7|
|  1820-11-01|  1|  44|   11|1820|      4|
|  1820-04-01|  1|  13|    4|1820|      7|
|  1821-02-01|  1|   5|    2|1821|      5|
|  1820-02-01|  1|   5|    2|1820|      3|
|  1820-12-01|  1|  48|   12|1820|      6|
|  1820-06-01|  1|  22|    6|1820|      5|
|  1821-01-01|  1|   1|    1|1821|      2|
+------------+---+----+-----+----+-------+



In [34]:
temperatures_table = temperatures_df.select(
        col('dt').alias('date'),
        month('dt').alias('month'),
        year('dt').alias('year'),
        col('AverageTemperature').alias('avg_temp'),
        col('AverageTemperatureUncertainty').alias('avg_temp_uncertainty'),
        col('City').alas('city'),
        col('Country').alas('country'),
        col('Latitude').alas('latitude'),
        col('Longitude').alas('longtitude')
    )

TypeError: 'Column' object is not callable